![QuantConnect Logo](https://cdn.quantconnect.com/web/i/icon.png)
<hr>

In [2]:
''' Objectives:
Derived form engulfing10.py
Identify volume spikes and highlight them on the graph
Plot volatility
Mark bars with higher than normal volatility
Calculate and plot stop market orders pre-emptively
Calculate entry points pre-emptively
Plot volatility inv
Plot volatility
Change default parameters
Plot ema
Calculate and plot number of stocks to hold
New ema crossover function
'''


import matplotlib.pyplot as plt
from datetime import datetime
import json
import pandas as pd
import math as m

# Predicts stocks move before earnings ------- #

# Returns X indexes of graph for datesh
def getXIndexes(dates, histDates):
    histDates = dict(zip(list(histDates.index), range(len(list(histDates.index)))) )
    ind = []
    keysList = histDates.keys()
    if(type(dates) != list):
        return histDates[min(keysList, key=lambda x: abs(x - dates))]
    for date in dates:
        if(date in histDates):
            ind.append(histDates[date])
            continue
        temp = min(keysList, key=lambda x: abs(x - date))
        ind.append(histDates[temp])
    print("Returning: {}".format(ind))
    return ind

# ax2 is not an optional argument. just to lazy to change that
def plot_candlestick(ax, history, highlight = False, stock = "____", ax2 = False, ax3 = False, ax4 = False): 
    previousSector = False

    indexes = dict(zip(list(history.index), range(len(list(history.index)))))

    ax.title.set_text("Start : {} End : {} ; Stock : {}".format(history.index[0], history.index[-1], stock.Symbol))


    # Plot closing price instead of closing bars
    x = [indexes[i.Index] for i in history.itertuples()]
    y = [i.close for i in history.itertuples()]
    ax.plot(x, y, c = "black", linewidth = 1)

    # Plot ema
    y = [i.exponentialmovingaverage for i in history.itertuples()]
    ax.plot(x, y, c = "blue")
    # Plot bands for volatility indicators
    y = [i.upperBand for i in history.itertuples()]
    ax3.plot(x, y, c = "orange")

    y = [i.rangeHigh for i in history.itertuples()]
    ax3.plot(x, y, c = "aqua")

    y = [i.rangeHighInv for i in history.itertuples()]
    ax2.plot(x, y, c = "aqua")

    y = [i.upperBandInv for i in history.itertuples()]
    ax2.plot(x, y, c = "orange")
    '''
    y = [i.open if i.close > i.open else i.close for i in history.itertuples()]
    c = ["palegreen" if i.close > i.open else "salmon" for i in history.itertuples()]
    h = [i.close - i.open if i.close > i.open else i.open - i.close for i in history.itertuples()]
    '''

    prevIndex = 0
    prevIndexDate = history.index[0]

    target = 10
    current = 0
    currentArray = {}
    # Plot bars
    for line in history.itertuples():
        x = indexes[line.Index]
        
        # Green bar
        if(line.close > line.open):
            c = "palegreen"
            y = line.open
            h = line.close - line.open
            yerr = [[h + line.open - line.low], [line.high - line.close]]
        # Red bar
        else:
            c = "salmon"
            y = line.close
            h = line.open - line.close
            yerr = [[h + line.close - line.low], [line.high - line.open]]

        if(highlight  != False):
            if(line.Index in highlight):
                if(c == "palegreen"):
                    c = "green"
                else:
                    c = "red"

        crossover, previousSector = emaCrossoverNew(line.Index, history, previousSector)
        if(crossover):
            print("Crossover at : {}".format(line.Index))
            ax.axvspan(prevIndex, x, facecolor = "red" if currentSector(line) else "green" , alpha = 0.2)
            # Plot number of stocks

            change = calcStockQuantity(prevIndexDate, line.Index, history, target, sell = not previousSector)
            current = current + change if previousSector else current - change
            if(current < 0):
                current = 0
            if(current > target):
                current = target

            currentArray[x] = current


            prevIndex = x
            prevIndexDate = line.Index

        # Plot bar
        #ax.bar(x, h, bottom = y, color = c, yerr = yerr, ecolor = "grey")
        ax4.bar(x, current, width = 1, color = "blue")
        # Plot the ax2 stuff (Selling points)
        barColor = "grey"
        if(calc_wvf(line, inv = True)):
            barColor = "lime"
        ax2.bar(x, line.wvfInv, width = 1, color = barColor)

        # Plot the ax3 stuff (Buying points)
        barColor = "grey"
        if(calc_wvf(line)):
            barColor = "lime"
        ax3.bar(x, line.wvf, width = 1, color = barColor)


    # Plot stock numbers
    #ax4.plot([x for x in currentArray], [currentArray[x] for x in currentArray], c = "orange")


# Tells how many stocks to buy or sell after each sector; called in plot_candlestick after ema crossover
def calcStockQuantity(startIndex, endIndex, history, target, sell = False):
    count = 0
    for i in history.loc[startIndex: endIndex].itertuples():
        if(calc_wvf(i, inv = True if sell else False)):
            count += 1
    return m.floor((count / (5 * 60)) * target)


# Tells whether wvf is peaking or not
def calc_wvf(currentBar, inv = False):
    if(inv):
        if(currentBar.wvfInv >= currentBar.rangeHighInv or currentBar.wvfInv >= currentBar.upperBandInv):
            return True
        return False

    if(currentBar.wvf >= currentBar.rangeHigh or currentBar.wvf >= currentBar.upperBand):
        return True
    return False

# Return True if ema crosses price
def emaCrossover(currentIndex, history):
    currentBar = history.loc[currentIndex]
    try:
        previousBar = history.loc[:currentIndex].iloc[-2]
    except:
        return False

    if(currentSector(currentBar) !=  currentSector(previousBar)):
        return True
    return False

# Return true if bullish sector and false if bearish sector
def currentSector(currentBar):
    if(currentBar.exponentialmovingaverage < currentBar.close):
        return True
    return False


def emaCrossoverNew(currentIndex, history, previousSector):
    currentBar = history.loc[currentIndex]
    try:
        previousBar = history.loc[:currentIndex].iloc[-2]
    except:
        return False, previousSector

    threshold = currentBar.close * 0.001
    if(previousSector):
        if(currentBar.close < currentBar.exponentialmovingaverage - threshold):
            previousSector = False
            return True, previousSector
    else:
        if(currentBar.close > currentBar.exponentialmovingaverage + threshold):
            previousSector = True
            return True, previousSector

    return False, previousSector


# Unused functions -----#
def calc_stopLoss(currentBar):
    wvf = min([currentBar.rangeHigh, currentBar.upperBand])

    return currentBar.closeHigh - (currentBar.closeHigh * wvf) / 100

def calc_entry(currentBar):
    wvf = min([ currentBar.rangeHighInv, currentBar.upperBandInv])
    return currentBar.closeLow + (currentBar.closeLow * wvf) / 100
# ----------------------#


def getPrediction(stock, benchmark, ecDate, today, plot = False, debug = False):

    # ----- Get historical data for requested stocks --------------------------------#
    historyBenchmark = qb.History(benchmark.Symbol, today - ecDate + timedelta(10), Resolution.Minute)
    history = qb.History(stock.Symbol, today - ecDate + timedelta(10), Resolution.Minute)
    history = history.loc[stock.Symbol].loc[ecDate:]
    # -------------------------------------------------------------------------------#

    # Initialise indicators ---------------#
    history['volumesma'] = history.volume.rolling(window = 10).mean()
    history['sma'] = history.close.rolling(window = 540).mean()
    # Calculate wvf

    pd = 1320
    lb = 3000
    bbl = 1200

    history['closeHigh'] = history.close.rolling(window = pd).max()
    history['wvf'] = ((history['closeHigh'] - history['low']) / history['closeHigh']) * 100
    history['rangeHigh'] = history['wvf'].rolling(window = lb).max() * 0.95
    
    history['upperBand'] = 2 * history['wvf'].rolling(window = bbl).std() + history['wvf'].rolling(window = bbl).mean() 
    history['wvfSma'] = history['wvf'].rolling(window = 9).mean()

    # Calculate wvf Inv
    history['closeLow'] = history.close.rolling(window = pd).min()
    history['wvfInv'] = ((history['high'] - history['closeLow'])/ history['closeLow'] ) * 100
    history['rangeHighInv'] = history['wvfInv'].rolling(window = lb).max() * 0.95
    history['upperBandInv'] = 2 * history['wvfInv'].rolling(window = bbl).std() + history['wvfInv'].rolling(window = bbl).mean() 
    
    # Input: period (int)
    atr = qb.Indicator(AverageTrueRange(120), stock.Symbol, today - ecDate + timedelta(10), Resolution.Minute)
    atr = atr.loc[ecDate:]

    roc = qb.Indicator(RateOfChange(120), stock.Symbol, today - ecDate + timedelta(10), Resolution.Minute)
    roc = roc.loc[ecDate:]

    ema = qb.Indicator(ExponentialMovingAverage(200), stock.Symbol, today - ecDate + timedelta(10), Resolution.Minute)
    ema = ema.loc[ecDate:]

    history = history.merge(ema, left_index = True, right_index = True)
    
    #history = history.loc[datetime(2020, 7, 2):datetime(2020, 7, 3)]
    # Print some trade stats header info --#
    if(debug):
        print("{}: From: {}; To: {}".format(stock.Symbol, ecDate, today))
    # -------


    # Identify engulfing pattern ----------#

    profits = 0
    losses = 0
    hightlightArray = []
    for currentBar in history.itertuples():
        # if first bar
        engulfing = calc_wvf(currentBar)


        if(engulfing):
            hightlightArray.append(currentBar.Index)
            width = abs(currentBar.close - currentBar.open)
            #print("{}: Upper : {} Lower : {} Time : {}".format(currentBar.close, currentBar.close + width, currentBar.close - width, currentBar.Index))





    # -------------------------------------#

    # -- PLOTTING STUFF -------------------#
    if(plot == True):
        fig, ax = plt.subplots(4,1,figsize=(30, 20), dpi=100, squeeze=False, sharex = True)
        fig.patch.set_facecolor('lightpink')
        plot_candlestick(ax[0,0], history, stock = stock, highlight = hightlightArray, ax2 = ax[1, 0], ax3 = ax[2, 0], ax4 = ax[3,0])
    # -------------------------------------#


    # -- Debugging stuff ------------------#
    if(debug == True):
        print("Profits: {} ; Losses: {}".format(profits, losses))
    # -------------------------------------#

    return profits, losses

# Algo setup ---------------------------------------------------------------------#
stocks = ['NET']

qb = QuantBook()
# Start date:
ecDate = datetime(2020, 7, 8)
# Today:
today = datetime(2020, 7, 17)
# End date: (# Start date is actually is end-date (bug)) today = setstartdate
qb.SetStartDate(2020, 7, 17)


benchmark = qb.AddEquity('SOLO')

profits = 0
losses = 0

for stock in stocks:
    stock = qb.AddEquity(stock)
    p, l = getPrediction(stock, benchmark, ecDate, today, debug = True, plot = True)
    profits += p
    losses += l

print("Total profits: {}; losses: {}".format(profits, losses))

# --------------------------------------------------------------------------------#